In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
import zipfile
import shutil

shutil.copy("/content/drive/MyDrive/DL/Dataset_train.zip","/content/Dataset_train.zip")
shutil.copy("/content/drive/MyDrive/DL/Dataset_test.zip","/content/Dataset_test.zip")

zip_path = "/content/Dataset_train.zip"
with zipfile.ZipFile(zip_path,"r") as zip_ref:
  zip_ref.extractall("./Dataset/")

import zipfile
zip_path = "/content/Dataset_test.zip"
with zipfile.ZipFile(zip_path,"r") as zip_ref:
  zip_ref.extractall("./Dataset_test/")


image_size = (1024, 1024)
batch_size = 64
IMG_SIZE =256


resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])



normalization_layer = tf.keras.layers.Rescaling(1./255)
resizing_layer = tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE)
flip_layer = tf.keras.layers.RandomRotation(0.1)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Dataset/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (resizing_layer(x), y))
train_ds = train_ds.map(lambda x, y: (flip_layer(x), y))


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Dataset/",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (resizing_layer(x), y))


test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/Dataset_test/",
    image_size=image_size,
    batch_size=batch_size,
)


test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (resizing_layer(x), y))



'/content/Dataset_test.zip'

In [ ]:
from tensorflow.keras.models import Model


import keras.backend as K
# def get_f1(y_true, y_pred): #taken from old keras source code
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3)

base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
    pooling=None,
    classes=4,
    classifier_activation="softmax",
)

83697664/83683744 [==============================] - 4s 0us/step


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/DL/xception_final/save_at_{epoch}.h5"),early_stopping_cb
]
epochs =20
model.compile(
    optimizer=keras.optimizers.Adam(1e-6),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/20
607/607 [==============================] - 525s 838ms/step - loss: 1.0942 - accuracy: 0.6926 - val_loss: 0.7107 - val_accuracy: 0.8955
Epoch 2/20
607/607 [==============================] - 501s 824ms/step - loss: 0.5230 - accuracy: 0.9025 - val_loss: 0.2986 - val_accuracy: 0.9198
Epoch 3/20
607/607 [==============================] - 500s 823ms/step - loss: 0.2570 - accuracy: 0.9350 - val_loss: 0.1687 - val_accuracy: 0.9567
Epoch 4/20
607/607 [==============================] - 502s 825ms/step - loss: 0.1513 - accuracy: 0.9642 - val_loss: 0.1003 - val_accuracy: 0.9850
Epoch 5/20
607/607 [==============================] - 500s 822ms/step - loss: 0.0958 - accuracy: 0.9822 - val_loss: 0.0608 - val_accuracy: 0.9915
Epoch 6/20
607/607 [==============================] - 500s 822ms/step - loss: 0.0653 - accuracy: 0.9879 - val_loss: 0.0399 - val_accuracy: 0.9940
Epoch 7/20
607/607 [==============================] - 501s 823ms/step - loss: 0.0465 - accuracy: 0.9906 - val_loss: 0.0283 -